In [1]:
from idmanagerclient import IDManager
from lodgatewayclient import LODGatewayClient
import json

prod_idm = IDManager("https://services.getty.edu/id-management")
rcv = LODGatewayClient("https://data.getty.edu/research/collections")

# pretty print
def pp(jsondoc):
    print(json.dumps(jsondoc, indent=2, sort_keys=True))

In [2]:
print(f"Connected to Production IDM (readonly): {prod_idm.ready}")
print(f"Connected to Production RCV (readonly): {rcv.ready}")

Connected to Production IDM (readonly): True
Connected to Production RCV (readonly): True


In [3]:
# Get a list of collection URIs from IDM (collections have accession numbers)
collections = [link['body']['id'] for link in prod_idm.query_links(body_generator="lodlvl2", motivation="contains-accno")]
collections[0]


'https://data.getty.edu/research/collections/component/311b7a02-04d1-52d5-ad8d-48c0f04ec86e'

In [4]:
collection_jsonld = rcv.get_item(collections[0])
print(collection_jsonld['_label'])

Getty Research Institute Scholars Program images and recordings


In [5]:
subjects = collection_jsonld['about']
pp(subjects[:4])

[
  {
    "_label": "Learning and scholarship",
    "id": "https://data.getty.edu/research/collections/document/9e6629c1-ea2c-5295-b064-27deead9731a",
    "type": "LinguisticObject"
  },
  {
    "_label": "Sound recordings",
    "id": "https://data.getty.edu/research/collections/document/53636583-c76a-576c-93ac-c5af73010088",
    "type": "LinguisticObject"
  },
  {
    "_label": "Congresses and conventions",
    "id": "https://data.getty.edu/research/collections/document/e23bbc93-42bd-5f97-ac46-8a86b919f60c",
    "type": "LinguisticObject"
  },
  {
    "_label": "Contact sheets",
    "id": "https://data.getty.edu/research/collections/document/344a51b2-d03a-5d2f-bcac-081bd10f0349",
    "type": "LinguisticObject"
  }
]


In [6]:
subject = rcv.get_item(subjects[0]['id'])
pp(subject)

{
  "@context": "https://linked.art/ns/v1/linked-art.json",
  "classified_as": [
    {
      "_label": "Subject Heading - Topical",
      "id": "https://data.getty.edu/local/thesaurus/aspace-subject-topical",
      "type": "Type"
    }
  ],
  "content": "Learning and scholarship",
  "format": "text/html",
  "id": "https://data.getty.edu/research/collections/document/9e6629c1-ea2c-5295-b064-27deead9731a",
  "identified_by": [
    {
      "classified_as": [
        {
          "_label": "generated URL slug",
          "id": "https://data.getty.edu/local/thesaurus/temporary-slug",
          "type": "Type"
        },
        {
          "_label": "Slug Identifier",
          "id": "https://data.getty.edu/local/thesaurus/slug-identifier",
          "type": "Type"
        }
      ],
      "content": "10YYA1",
      "id": "https://data.getty.edu/research/collections/document/9e6629c1-ea2c-5295-b064-27deead9731a/slug",
      "type": "Identifier"
    }
  ],
  "part": [
    {
      "id": "https:

In [7]:
# What resources does this subject get applied to?
results = rcv.sparql("select ?g where { GRAPH ?g {?s ?p <" + subject['id'] + "> .}} LIMIT 100")

# Get the label of each thing and print it
# (Deal with the SPARQL SELECT JSON response:)
for r in results['results']['bindings']:
    uri = r['g']['value']
    resource = rcv.get_item(uri)
    print(f"{uri} - {resource.get('_label')}")

https://data.getty.edu/research/collections/component/311b7a02-04d1-52d5-ad8d-48c0f04ec86e - Getty Research Institute Scholars Program images and recordings
https://data.getty.edu/research/collections/component/88e169d7-a535-5e74-a60f-a1ead22496a1 - Guest scholar and conservator files
